In [1]:
from pianoq.misc.mplc_writeup_imports import *
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_10_30_make_mirror_work'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

# WFM 

In [6]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_U1U_dead_zone_450pix_dz10_8mm.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_U1U_dead_zone_450pix_dz10_8mm.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=450e-6,
                                    last_plane_extra_dist=8e-3)


{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087, 0.087, 0.087, 0.087, 0.138, 0.087, 0.087, 0.087, 0.087,
       0.095]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}


100%|██████████| 15/15 [02:52<00:00, 11.48s/it]


In [7]:
mplc_sim.res.saveto(mplc_res_path, smaller=True)
mplc_sim.res.show_overlap()
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9885901232826865

In [9]:
mplc_sim.res.show_all(2)

# Hardware

In [2]:
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=35)
integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

God timetagger


In [3]:
backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)


Got Zaber motors
Got Thorlabs motors


# Single counts

In [6]:
# temp phases, that are probably better than nothing 
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"
masks_path = fr'{dir_path}\2024_10_30_13_10_41_U1U_dead_zone_450pix_dz10_8mm.masks'

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)
time.sleep(0.5)
mplc.restore_location()

scan_saveto_path = rf'{dir_path}\{tnow()}_5spots_fine_signal_dead_zone_450um_dz10_8mm_p10_tilts_35.scan'

start_x = 12.75
end_x = 13.35
start_y = 4.75
end_y = 6.6

resolution = 1
x_pixels = 24 // resolution
y_pixels = 76 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path) 
tt = get_timetagger(integration_time=1)  # important to have a new tt 
t1 = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)

time.sleep(2)

scan_saveto_path = rf'{dir_path}\{tnow()}_5spots_fine_idler_dead_zone_450um_dz10_8mm_p10_tilts_35.scan'

start_x = 9.25
end_x = 9.75
start_y = 3.6
end_y = 5.35

x_pixels = 20 // resolution
y_pixels = 70 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)
tt2 = get_timetagger(integration_time=1)
t2 = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt2)
t1.join()
send_telegram_message('Done one singles scan')
t2.join()    
send_telegram_message('Done second singles scan')

{'ok': True,
 'result': {'message_id': 17,
  'from': {'id': 7897393437,
   'is_bot': True,
   'first_name': 'complexphoton',
   'username': 'complexphoton_bot'},
  'chat': {'id': 476169345,
   'first_name': 'Ronen',
   'last_name': 'Shekel',
   'username': 'RonenShekel',
   'type': 'private'},
  'date': 1730292550,
  'text': 'Done second singles scan'}}